In [5]:
pool_type = ['maxavg', 'max']
hidden_layer_type = [True, False] # add one hidden layer or not
dropout_rate_type = [0.2, 0.3, 0.4]
lr_type = [0.001, 0.005, 0.01]
scheduler_type = [True, False] # use Cosine Annealing or not
opt_type = ['SGD', 'Adam'] # optimizer

In [6]:
for i in range(12):
    (share, remainder) = divmod(i, len(dropout_rate_type))
    dropout_rate = dropout_rate_type[remainder]
    (share, remainder) = divmod(share, len(hidden_layer_type))
    hidden_layer = hidden_layer_type[remainder]
    (share, remainder) = divmod(share, len(pool_type))
    pooling = pool_type[remainder]
    print(pooling, hidden_layer, dropout_rate)

maxavg True 0.2
maxavg True 0.3
maxavg True 0.4
maxavg False 0.2
maxavg False 0.3
maxavg False 0.4
max True 0.2
max True 0.3
max True 0.4
max False 0.2
max False 0.3
max False 0.4


In [6]:
a = 7
b = 3
(share, remainder) = divmod(a,b)

In [4]:
name = 'asdf'

with open("./results/hyperparameter_experiments/"+name+'.txt', "a") as file:
    file.write('TF : ')
    file.write(name)
    file.write(" - AUC Test : ")
file.close()

In [20]:
import torch

In [35]:
wNeu = torch.randn(33,1)
print(wNeu.size())
wNeu

torch.Size([33, 1])


tensor([[ 0.0983],
        [-2.6496],
        [-1.3451],
        [-0.7911],
        [-1.6466],
        [ 0.5343],
        [-0.9837],
        [ 1.5457],
        [ 1.8328],
        [ 1.6524],
        [-0.4289],
        [-0.2520],
        [-0.9636],
        [-0.7389],
        [ 0.0792],
        [ 0.1097],
        [ 0.4058],
        [ 2.4089],
        [-0.6629],
        [-0.4676],
        [ 1.3670],
        [-0.4338],
        [-0.5098],
        [-2.2858],
        [-2.7613],
        [ 1.2614],
        [-0.7054],
        [ 0.9729],
        [ 1.5633],
        [ 0.6913],
        [ 0.2943],
        [-0.4785],
        [-0.4235]])

In [36]:
torch.nn.init.xavier_uniform_(wNeu)

tensor([[-0.2965],
        [ 0.2922],
        [ 0.2110],
        [-0.4146],
        [-0.3608],
        [-0.1170],
        [-0.1165],
        [ 0.0375],
        [-0.0476],
        [-0.2089],
        [-0.2579],
        [ 0.3399],
        [ 0.4025],
        [-0.3378],
        [-0.1743],
        [-0.1877],
        [ 0.2006],
        [-0.1398],
        [ 0.3025],
        [-0.1040],
        [ 0.3120],
        [ 0.1359],
        [ 0.1609],
        [-0.2765],
        [ 0.3999],
        [ 0.0303],
        [-0.1843],
        [ 0.3718],
        [-0.3237],
        [-0.1602],
        [ 0.2782],
        [ 0.1124],
        [-0.0359]])

In [37]:
(wNeu, wNeuBias) = wNeu.split(list(wNeu.size())[0]-1, dim=0)

In [38]:
print(wNeu.size())
print(wNeuBias.size())

torch.Size([32, 1])
torch.Size([1, 1])


In [39]:
wNeuBias.size()

torch.Size([1, 1])

In [41]:
import os
import time
import datetime

import torch
import random
import argparse
import numpy as np
import pandas as pd
import torch.nn.functional as F

from sklearn import metrics
from utils import logsampler, sqrtsampler, datasets, dataset_loader, test_dataset_loader
from network import ConvNet, ConvNet_test

 
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [42]:
path = './data/encode/'
all_dataset_names = datasets(path)

In [43]:
all_dataset_names

[['./data/encode/ARID3A_K562_ARID3A_(sc-8821)_Stanford_AC.seq.gz',
  './data/encode/ARID3A_K562_ARID3A_(sc-8821)_Stanford_B.seq.gz'],
 ['./data/encode/CTCFL_K562_CTCFL_(SC-98982)_HudsonAlpha_AC.seq.gz',
  './data/encode/CTCFL_K562_CTCFL_(SC-98982)_HudsonAlpha_B.seq.gz'],
 ['./data/encode/ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq.gz',
  './data/encode/ELK1_GM12878_ELK1_(1277-1)_Stanford_B.seq.gz'],
 ['./data/encode/FOXA1_HepG2_FOXA1_(SC-101058)_HudsonAlpha_AC.seq.gz',
  './data/encode/FOXA1_HepG2_FOXA1_(SC-101058)_HudsonAlpha_B.seq.gz'],
 ['./data/encode/GABPA_GM12878_GABP_HudsonAlpha_AC.seq.gz',
  './data/encode/GABPA_GM12878_GABP_HudsonAlpha_B.seq.gz'],
 ['./data/encode/MYC_H1-hESC_c-Myc_Stanford_AC.seq.gz',
  './data/encode/MYC_H1-hESC_c-Myc_Stanford_B.seq.gz'],
 ['./data/encode/REST_GM12878_NRSF_HudsonAlpha_AC.seq.gz',
  './data/encode/REST_GM12878_NRSF_HudsonAlpha_B.seq.gz'],
 ['./data/encode/SP1_GM12878_SP1_HudsonAlpha_AC.seq.gz',
  './data/encode/SP1_GM12878_SP1_HudsonAlpha_B.se

In [46]:
for i in range(len(all_dataset_names)):
    dataset_name = all_dataset_names[i]
    train_dataset_path = dataset_name[0]
    test_dataset_path = dataset_name[1]
    name = train_dataset_path.split(path)[1].split("_AC")[0]
    createFolder('./Models/'+name)    

In [47]:
f = 0.001

In [49]:
print(str(round(f, 6)))

0.001


In [3]:
for i in range(1):
    print(str(i+1))

1
